In [66]:
"""
FOR GRID INDEXING TASK ONLY
"""


import json
import cv2
import numpy as np
import os
from tqdm import tqdm

H = 900
W = 1600
patchsize_H = 100
patchsize_W = 100

before_mask = "/mnt/bn/nlhei-nas/liubangya/proj/vlm-found3d/vlm/QA/pairs-indexing-test/QA_pairs_qwen.train.json"

img_dump = before_mask.replace(".json", "_masked")
if os.path.exists(img_dump):
    # remove the dir
    os.system(f"rm -rf {img_dump}")
os.makedirs(img_dump, exist_ok=True)

with open(before_mask, "r") as f:
    qas = json.load(f)

def coloring(rgb, channel, alpha=0.5):
    if channel == 2:
        new_rgb = rgb * alpha + 255 * (1 - alpha)
        # uint8-fy
        return new_rgb.astype(np.uint8)
    else:
        new_rgb = rgb * alpha
        # uint8-fy
        return new_rgb.astype(np.uint8)


new_qas = []
for qa in tqdm(qas):
    img_name = qa["image"][0]
    post_fix = img_name.split(".")[-1]
    new_img_name = os.path.join(img_dump, f"{qa['id']}.{post_fix}")
    gt = qa["conversations"][-1]["value"]
    gt = gt.replace("```json", "").replace("```", "")
    gt = json.loads(gt)
    row = gt["row"]
    col = gt["col"]
    img = cv2.imread(img_name)
    img = cv2.resize(img, (W, H))
    # coloring the patch into red
    alpha = 0.5
    
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0], 0)
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1], 1)
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2], 2)
    cv2.imwrite(new_img_name, img)
    new_qa = qa.copy()
    new_qa["image"] = [new_img_name]
    new_qas.append(new_qa)

after_mask = before_mask.replace(".json", ".masked.json")
with open(after_mask, "w") as f:
    json.dump(new_qas, f, indent=2)

    

100%|██████████| 800/800 [00:22<00:00, 35.21it/s]
